In [1]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np
from io import StringIO
import json

In [4]:
import json
import re
import pandas as pd
from bs4 import BeautifulSoup

def parse_html_table(html):
    """Parse one HTML table into DataFrame with Date, Subject, Remarks, Link."""
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if not table:
        return None

    rows = []
    for tr in table.select("tbody tr"):
        cols = tr.find_all("td")
        if len(cols) < 3:
            print("There are less than three cols, skipping")
            continue

        date = cols[0].get_text(strip=True)
        remarks = cols[2].get_text(strip=True)

        a = cols[1].find("a")
        subject = a.get_text(" ", strip=True) if a else cols[1].get_text(strip=True)
        link = ""
        if a and a.get("onclick"):
            onclick = a["onclick"]
            if "newwindow1" in onclick:
                start = onclick.find("'") + 1
                end = onclick.rfind("'")
                link = onclick[start:end]

        rows.append([date, subject, remarks, link])

    return pd.DataFrame(rows, columns=["Date", "Subject", "Remarks", "Link"])

def clean_sheet_name(name, fallback="Sheet"):
    safe = re.sub(r'[\[\]\:\*\?\/\\]', "_", str(name))
    return safe[:31] if safe else fallback

def cache_to_excel(cache_file, excel_out="Orders_All.xlsx"):
    with open(cache_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    used_names = {}  # track counts per sheet name

    with pd.ExcelWriter(excel_out, engine="xlsxwriter") as writer:
        for scraped in data["records"][0]["scraped_data"]:
            if not scraped.get("data_present"):
                continue

            action = scraped.get("action", "unknown")
            webpage = scraped.get("webpage", "")
            for resp in scraped.get("response", []):
                if resp.get("type") != "table_html":
                    continue

                df = parse_html_table(resp["value"])
                if df is None or df.empty:
                    continue

                titles = resp.get("title", [])
                base_name = clean_sheet_name(titles[-1] if titles else action)

                count = used_names.get(base_name, 0) + 1
                used_names[base_name] = count
                sheet_name = base_name if count == 1 else clean_sheet_name(f"{base_name}_{count}")


                meta_rows = [["Action", action], ["Webpage", webpage]]
                for i, t in enumerate(titles, start=1):
                    meta_rows.append([f"Header{i}", t])
                meta = pd.DataFrame(meta_rows, columns=["Meta", "Value"])
                meta.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

                df_visible = df[["Date", "Subject", "Remarks","Link"]]
                startrow = len(meta) + 2
                df_visible.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)

                # ws = writer.sheets[sheet_name]
                # for i, (txt, url) in enumerate(zip(df["Subject"], df["Link"]), start=startrow+1):
                #     if url:
                #         ws.write_url(i, 1, url, string=txt)

    print(f"Excel saved: {excel_out}")


# Example usage
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-23\25-09-23T15-32-09_cache.json"
cache_to_excel(path, "Orders_example_All.xlsx")


There are less than three cols, skipping
Excel saved: Orders_example_All.xlsx


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.sundarammutual.com/mutual-fund-commission-disclosure")

links = driver.find_elements(By.CSS_SELECTOR,".d-block")

# print(vehicle_dashboard.get_attribute("outerHTML"))

for link in links:
    print(link.get_attribute("outerHTML"))
"pnl_trans","pnl_revenue","pnl_permit","pnl_taxDef"
time.sleep(5)
driver.quit()


In [ ]:
import requests
import pandas as pd
from io import BytesIO,StringIO

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})

def fetch_rba():
    url = "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    return pd.read_excel(BytesIO(r.content))

def fetch_bcra():
    url = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    r = session.get(url, timeout=15, verify=False)
    r.raise_for_status()
    return pd.read_html(r.text)[0]


def fetch_banxico(token, series_ids, start, end):
    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{','.join(series_ids)}/datos/{start}/{end}"
    headers = {"Bmx-Token": token}
    r = session.get(url, headers=headers, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for s in data['bmx']['series']:
        sid = s['idSerie']
        rec = {d['fecha']: d['dato'] for d in s['datos']}
        records[sid] = rec
    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns, dayfirst=True)
    return df.sort_index(axis=1)


def fetch_boc(series_ids, start, end):
    url = f"https://www.bankofcanada.ca/valet/observations/{','.join(series_ids)}/json?start_date={start}&end_date={end}"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for row in data["observations"]:
        date = row["d"]
        for sid in series_ids:
            records.setdefault(sid, {})[date] = row.get(sid, {}).get("v")
    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns)
    return df.sort_index(axis=1)


with pd.ExcelWriter("central_banks.xlsx") as writer:
    token = "1846d489e18513d4306b70e257a21edb56e82325de0139af91dd2176975029bf"
    fetch_banxico(token, ["SF61745","SF331451","SF43783","SF43878","SF111916"], "2025-09-18", "2025-09-22")\
        .to_excel(writer, sheet_name="Banxico")
    fetch_boc(["V39079","CL.CDN.MOST.1DL","V39078","V80691310","V122530"], "2025-09-18", "2025-09-23")\
        .to_excel(writer, sheet_name="BoC")
    fetch_rba().to_excel(writer, sheet_name="RBA", index=False)
    fetch_bcra().to_excel(writer, sheet_name="BCRA", index=False)


In [16]:
"Title	Cash Rate Target	Change in the Cash Rate Target 	Interbank Overnight Cash Rate	Highest Interbank Overnight Cash Rate	Lowest Interbank Overnight Cash Rate	Volume of Cash Market Transactions	Number of Cash Market Transactions	Total Return Index	EOD 1-month BABs/NCDs	EOD 3-month BABs/NCDs	EOD 6-month BABs/NCDs	1-month OIS	3-month OIS	6-month OIS	1-month Treasury Note	3- month Treasury Note	6- month Treasury Note".split("\t")

['Title',
 'Cash Rate Target',
 'Change in the Cash Rate Target ',
 'Interbank Overnight Cash Rate',
 'Highest Interbank Overnight Cash Rate',
 'Lowest Interbank Overnight Cash Rate',
 'Volume of Cash Market Transactions',
 'Number of Cash Market Transactions',
 'Total Return Index',
 'EOD 1-month BABs/NCDs',
 'EOD 3-month BABs/NCDs',
 'EOD 6-month BABs/NCDs',
 '1-month OIS',
 '3-month OIS',
 '6-month OIS',
 '1-month Treasury Note',
 '3- month Treasury Note',
 '6- month Treasury Note']

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
path = r"C:\Users\rando\Office Projects\outputs\scrape_output\cache\2025-09-23\25-09-23T22-31-06_cache.json"
data = Helper.load_json(path)
OperationExecutor.generate_cache_doc_report(data)

In [ ]:
import requests
import pandas as pd
from io import BytesIO,StringIO
from datetime import datetime

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})

def fetch_rba(): 
    url = "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx" 
    r = session.get(url, timeout=15) 
    r.raise_for_status() 
    df = pd.read_excel(BytesIO(r.content)) 
    df.columns = df.iloc[1, :]
    df = df.iloc[-5:, :].T.reset_index()
    df.columns = df.iloc[0, :]
    df = df.drop(df.index[0])
    df.columns = [(lambda c: pd.to_datetime(c, errors="ignore"))(c) for c in df.columns]
    df.columns = [c.strftime("%m-%d-%Y") if isinstance(c, pd.Timestamp) else c for c in df.columns]
    
    return df

def fetch_bcra():
    url = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    r = session.get(url, timeout=15, verify=False)
    r.raise_for_status()
    df = pd.read_html(r.text)[0]
    
    today = datetime.today().strftime("%d-%m-%Y")
    date_range = pd.date_range(end=today, periods=5)   # datetime index

    df.columns = ["Header","Date","Value"]
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)

    reshaped = (df.pivot_table(index="Header", columns="Date", values="Value", aggfunc="first"))
    reshaped = reshaped.reindex(columns=date_range)
    reshaped.columns = [d.strftime("%m-%d-%Y") for d in reshaped.columns]
    reshaped = reshaped.reset_index()
    return reshaped

def fetch_banxico(token, series_ids, start, end): 
    #mapper
    series_map = {
        "SF61745": "Target rate 1/",
        "SF331451": "Overnight TIIE funding rate 2/",
        "SF43783": "28 day TIIE 3/",
        "SF43878": "91 day TIIE 3/",
        "SF111916": "182 day TIIE 3/"
    }
    
    #api call
    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{','.join(series_ids)}/datos/{start}/{end}"
    headers = {"Bmx-Token": token}
    r = session.get(url, headers=headers, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for s in data['bmx']['series']:
        sid = s['idSerie']
        rec = {d['fecha']: d['dato'] for d in s['datos']}
        records[sid] = rec
    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns, dayfirst=True)
    
    df.index = df.index.map(lambda x: series_map.get(x, x))
    # print(df)
    df.columns = [d.strftime("%m-%d-%Y") for d in df.columns]
    
    return df.sort_index(axis=1)

def fetch_boc(series_ids, start, end):
    series_map = {
        "V39079": "Target for the overnight rate",
        "CL.CDN.MOST.1DL": "Overnight money market financing rate1",
        "V39078": "Bank rate - Daily",
        "V80691310": "Bank rate - Weekly",
        "V122530": "Bank rate - Monthly"
    }
    
    url = f"https://www.bankofcanada.ca/valet/observations/{','.join(series_ids)}/json?start_date={start}&end_date={end}"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for row in data["observations"]:
        date = row["d"]
        for sid in series_ids:
            records.setdefault(sid, {})[date] = row.get(sid, {}).get("v")

    df = pd.DataFrame.from_dict(records, orient="index")
    # make sure column labels are datetime
    df.columns = pd.to_datetime(df.columns)
    full_range = pd.date_range(start=start, end=end, freq="D")
    df = df.reindex(columns=full_range)
    df.columns = [d.strftime("%m-%d-%Y") for d in df.columns]
    df.index = df.index.map(lambda x: series_map.get(x, x))

    return df.sort_index(axis=1)


with pd.ExcelWriter("central_banks_1.xlsx") as writer:
    token = "1846d489e18513d4306b70e257a21edb56e82325de0139af91dd2176975029bf"
    fetch_banxico(token, ["SF61745","SF331451","SF43783","SF43878","SF111916"], "2025-09-25", "2025-09-29")\
        .to_excel(writer, sheet_name="Banxico")
    fetch_boc(["V39079","CL.CDN.MOST.1DL","V39078","V80691310","V122530"], "2025-09-25", "2025-09-29")\
        .to_excel(writer, sheet_name="BoC")
    fetch_rba().to_excel(writer, sheet_name="RBA", index = False)
    fetch_bcra().to_excel(writer, sheet_name="BCRA", index=False)

In [ ]:
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime, timedelta

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})


# ---- Shared Pandas Helpers ----
def normalize_columns_to_dates(df):
    """Convert datetime-like columns to mm-dd-YYYY strings."""
    df.columns = [
        c.strftime("%m-%d-%Y") if isinstance(c, pd.Timestamp) else c
        for c in df.columns
    ]
    return df


def ensure_full_date_range(df, start, end):
    """Ensure dataframe has full date coverage, fill missing dates with NaN."""
    full_range = pd.date_range(start=start, end=end, freq="D")
    df = df.reindex(columns=full_range)
    return normalize_columns_to_dates(df)

# ---- Fetchers ----
def fetch_rba(cfg, start=None, end=None):
    r = session.get(cfg["url"], timeout=15)
    r.raise_for_status()
    df = pd.read_excel(BytesIO(r.content))

    df.columns = df.iloc[1, :]
    df = df.iloc[-5:, :].T.reset_index()
    df.columns = df.iloc[0, :]
    df = df.drop(df.index[0])

    return normalize_columns_to_dates(df)


def fetch_bcra(cfg, start=None, end=None):
    r = session.get(cfg["url"], timeout=15, verify=False)
    r.raise_for_status()
    df = pd.read_html(r.text)[0]

    today = datetime.today()
    date_range = pd.date_range(end=today, periods=5)   # last 5 days

    df.columns = ["Header", "Date", "Value"]
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)

    reshaped = df.pivot_table(index="Header", columns="Date", values="Value", aggfunc="first")
    reshaped = reshaped.reindex(columns=date_range)
    reshaped = normalize_columns_to_dates(reshaped)
    return reshaped.reset_index()


def fetch_banxico(cfg, start, end):
    url = f"{cfg['url']}/{','.join(cfg['series_ids'])}/datos/{start}/{end}"
    r = session.get(url, headers=cfg["headers"], timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for s in data['bmx']['series']:
        sid = s['idSerie']
        rec = {d['fecha']: d['dato'] for d in s['datos']}
        records[sid] = rec

    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns, dayfirst=True)
    df.index = df.index.map(lambda x: cfg["series_map"].get(x, x))

    return normalize_columns_to_dates(df.sort_index(axis=1))


def fetch_boc(cfg, start, end):
    url = f"{cfg['url']}/{','.join(cfg['series_ids'])}/json?start_date={start}&end_date={end}"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    data = r.json()

    records = {}
    for row in data["observations"]:
        date = row["d"]
        for sid in cfg["series_ids"]:
            records.setdefault(sid, {})[date] = row.get(sid, {}).get("v")

    df = pd.DataFrame.from_dict(records, orient="index")
    df.columns = pd.to_datetime(df.columns)
    df = ensure_full_date_range(df, start, end)
    df.index = df.index.map(lambda x: cfg["series_map"].get(x, x))

    return df.sort_index(axis=1)

CONFIG = {
    "Banxico": {
        "fetcher": fetch_banxico,
        "url": "https://www.banxico.org.mx/SieAPIRest/service/v1/series",
        "headers": {"Bmx-Token": "1846d489e18513d4306b70e257a21edb56e82325de0139af91dd2176975029bf"},
        "series_ids": ["SF61745","SF331451","SF43783","SF43878","SF111916"],
        "series_map": {
            "SF61745": "Target rate 1/",
            "SF331451": "Overnight TIIE funding rate 2/",
            "SF43783": "28 day TIIE 3/",
            "SF43878": "91 day TIIE 3/",
            "SF111916": "182 day TIIE 3/"
        }
    },
    "BoC": {
        "fetcher": fetch_boc,
        "url": "https://www.bankofcanada.ca/valet/observations",
        "series_ids": ["V39079","CL.CDN.MOST.1DL","V39078","V80691310","V122530"],
        "series_map": {
            "V39079": "Target for the overnight rate",
            "CL.CDN.MOST.1DL": "Overnight money market financing rate1",
            "V39078": "Bank rate - Daily",
            "V80691310": "Bank rate - Weekly",
            "V122530": "Bank rate - Monthly"
        }
    },
    "RBA": {
        "fetcher": fetch_rba,
        "url": "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx"
    },
    "BCRA": {
        "fetcher": fetch_bcra,
        "url": "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    }
}

def run_pipeline(days=5, outfile="central_banks_config.xlsx"):
    end = datetime.today()
    start = end - timedelta(days=days)

    with pd.ExcelWriter(outfile) as writer:
        for name, cfg in CONFIG.items():
            fetcher = cfg["fetcher"]

            if name in ["Banxico", "BoC"]:
                df = fetcher(cfg, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"))
            else:
                df = fetcher(cfg, start, end)

            df.to_excel(writer, sheet_name=name, index=not name in ["RBA", "BCRA"])
    print(f"✅ Data saved to {outfile}")


if __name__ == "__main__":
    run_pipeline(days=5)



In [3]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-26\25-09-26T15-44-07_cache.json"
# path = r"25-09-08T18-33-09_cache.json5"
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-26\25-09-26T15-39-53_cache.json"

    
root_file = os.path.basename(path).split("_")[0]
function_to_execute = {
"primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
"secondary":[["sha1","norm_table","SHA_ONE"]],
}
try:
    data = Helper.load_json(path, typ="json5")
    processed_data = ops.runner(data,function_to_execute)
    Helper.save_json(processed_data,f"{root_file}_process.json")
except Exception as e:
    print(f"\nError in Processing.. Skipping, {e}")


>>Processing Bank of Baroda


In [4]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

v1 = r"25-09-26T15-44-07_process.json"
v2 = r"25-09-26T15-39-53_process.json"
yesterday = Helper.load_json(v2)
today = Helper.load_json(v1)

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-080925.json")

ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\1109_1009_compare\compare-080925.json"
result = Helper.load_json(path)
ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'